In [30]:
import warnings
import optuna
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore', category=UserWarning)

## Demo 1
Find the value of x that minimizes the function: f(x) = (x - 2)^2 + 1

In [ ]:
# Find the value of x that minimizes the function: f(x) = (x - 2)^2 + 1

def objective(trial):
    pass

# Create and run study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
print(f"{study.best_value = }")
print(f"{study.best_params = }")

## Exercise 1
Find the values of x and y that mininize the function: f(x,y) = (x - 2)^2 + (y + 3)^2 + 1

In [ ]:
def objective(trial):
    pass

# Create and run study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

## Visualization

In [ ]:
# Visualize the optimization process

# Get optimization history
optuna.visualization.plot_optimization_history(study)


In [ ]:
# You can also visualize parameter importance
optuna.visualization.plot_param_importances(study)


## Optimize LightGBM estimator

### Prepare Dataset

In [27]:
# Load and prepare data
data = load_breast_cancer()
print(f"Dataset shape: {data.data.shape}")  # 569 samples, 30 features
print("\nTarget distribution:")
print(f"Malignant (0): {(data.target == 0).sum()}")
print(f"Benign (1): {(data.target == 1).sum()}")
print("\nFeatures:", data.feature_names.tolist())

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.2, random_state=42
)
feature_names = data.feature_names.tolist()
X_train = pd.DataFrame(X_train, columns=feature_names)
X_test = pd.DataFrame(X_test, columns=feature_names)

Dataset shape: (569, 30)

Target distribution:
Malignant (0): 212
Benign (1): 357

Features: ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']


In [31]:
def objective(trial):
    # Define the hyperparameters to optimize
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'num_leaves': trial.suggest_int('num_leaves', 20, 50),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'verbose': -1  # Add this to suppress LightGBM logger

    }
    
    # Create model with suggested hyperparameters
    model = LGBMClassifier(**params, random_state=42)
    
    # Use cross-validation to evaluate the model
    cv_scores = cross_val_score(
        model, 
        X_train, 
        y_train, 
        cv=5,
        scoring='accuracy'
    )
    
    # Return mean CV score
    return 1.0 - cv_scores.mean()  # Minimize 1-accuracy

# Create and run study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2025-02-22 11:04:39,005] A new study created in memory with name: no-name-137bc311-9ea1-4f42-8229-3b54702cefa8
[I 2025-02-22 11:04:39,375] Trial 0 finished with value: 0.03736263736263745 and parameters: {'learning_rate': 0.027088305217947986, 'n_estimators': 825, 'num_leaves': 23, 'max_depth': 8, 'min_child_samples': 53, 'subsample': 0.7261882448689871, 'colsample_bytree': 0.9102264116623434}. Best is trial 0 with value: 0.03736263736263745.
[I 2025-02-22 11:04:39,443] Trial 1 finished with value: 0.02637362637362628 and parameters: {'learning_rate': 0.0719234399264858, 'n_estimators': 133, 'num_leaves': 36, 'max_depth': 7, 'min_child_samples': 95, 'subsample': 0.9267079317303797, 'colsample_bytree': 0.6379166434824188}. Best is trial 1 with value: 0.02637362637362628.
[I 2025-02-22 11:04:39,632] Trial 2 finished with value: 0.02857142857142847 and parameters: {'learning_rate': 0.06177439300767418, 'n_estimators': 605, 'num_leaves': 21, 'max_depth': 6, 'min_child_samples': 89, 'sub

In [37]:
print(f"{study.best_params}")
print(f"{(1 - study.best_value) * 100:.2f}%")

{'learning_rate': 0.06289773696575433, 'n_estimators': 337, 'num_leaves': 35, 'max_depth': 3, 'min_child_samples': 88, 'subsample': 0.8505726598555903, 'colsample_bytree': 0.7299723179434713}
97.80%
FrozenTrial(number=21, state=1, values=[0.0219780219780219], datetime_start=datetime.datetime(2025, 2, 22, 11, 4, 42, 884446), datetime_complete=datetime.datetime(2025, 2, 22, 11, 4, 43, 22044), params={'learning_rate': 0.06289773696575433, 'n_estimators': 337, 'num_leaves': 35, 'max_depth': 3, 'min_child_samples': 88, 'subsample': 0.8505726598555903, 'colsample_bytree': 0.7299723179434713}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.1, log=False, low=0.01, step=None), 'n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'num_leaves': IntDistribution(high=50, log=False, low=20, step=1), 'max_depth': IntDistribution(high=8, log=False, low=3, step=1), 'min_child_samples': IntDistribution(high=100, log=Fal

## Exercise 2:

In [ ]:
import optuna
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

scaler = StandardScaler()
# Using the same breast cancer dataset defined above
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate=0.1):
        super(SimpleNet, self).__init__()
        self.layer = nn.Linear(input_size, hidden_size)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.output = nn.Linear(hidden_size, 2)
        
    def forward(self, x):
        x = self.layer(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

def train_model(model, train_loader, criterion, optimizer, device='cpu'):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate_model(model, test_loader, device='cpu'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
    return correct / total

def objective(trial):
    # Student Task 1: Define hyperparameters to optimize
    # Use trial.suggest_float, trial.suggest_int, etc.
    # Example format:
    # hidden_size = trial.suggest_int('hidden_size', min_value, max_value)
    # learning_rate = 
    # dropout_rate = 
    # batch_size = 
    # n_epochs = 
    
    # Create model and move to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SimpleNet(input_size=20, 
                     hidden_size=hidden_size, 
                     dropout_rate=dropout_rate).to(device)
    
    # Create data loaders
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(n_epochs):
        train_loss = train_model(model, train_loader, criterion, optimizer, device)
        accuracy = evaluate_model(model, test_loader, device)
        
        # Enable pruning (early stopping) based on accuracy
        trial.report(accuracy, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return accuracy



In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, timeout=60)

# TODO:
# Print best trial information
# Create visualizations